In [2]:
#Follow along exercises with http://neuralnetworksanddeeplearning.com/chap1.html
print("yo")

yo


Does this render in github?


Notes:
Perceptrons sufficient for universal computation because the can implement NAND. Kinda cool way to introduce them

$$w \dot x = \sum w_j x_j$$

Exercise: NAND gate for addition
TODO: hack latex drawing here
x1 x2  a == NAND(x1,x2) b == NAND(x1,a) c== NAND(x2,a) low_bit == NAND(b,c) carry_bit== NAND(a,a)  result   base10
0  0        1            1                  1               0                   0                   00          0
0  1        1            1                  0               1                   0                   01          1
1  1        0            1                  1               0                   1                   10          2

Since Perceptrons can implment NAND, the can do this too
